### Goal and description

In [1]:
### This script connects to IBKR on a daily basis. If there is no data, then it will catch
### the latest data and ingest it to csv. It will add this to the master file

### Imports

In [2]:
# Catch forex data
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from ib_insync import *
import pandas as pd
import logging
import os
from datetime import date
from datetime import datetime as datetime
import numpy as np


### Parameters

In [3]:
#Reqhistoricaldata
p_endDateTime = ''
p_durationStr = '10 Y'
p_barSizeSetting='1 day'
p_whatToShow='MIDPOINT'
p_useRTH= True
p_formatDate=1
p_keepUpToDate=True
p_contract = 'EURUSD'
today = str(date.today())
convert_string_to_datetime = lambda x: datetime.strptime(x,"%Y-%m-%d")

directory_empty = False

#file_path = p_contract
p_file_path =  r"C:\Users\hugo.lu\repos\repos_backup\ib_insync\Raw data\\" + p_contract + "\\"

### Definitions

In [4]:
def onBarUpdate(*args):
    plt.close()
    plot = util.barplot(bars)
    clear_output(wait=True)
    display(plot)
    global df
    df = util.df(bars)
    # print(df)

### See what is available

In [5]:
data_list = os.listdir(path=p_file_path)
data_sub_list = []
### Check what is in the directory
if len(data_list) == 0:
    print("Nothing in directory")
    directory_empty = True
else:
    for item in data_list:
        try:
            data_sub_list.append(datetime.strptime(item.split("_")[0],"%Y-%m-%d"))
        except:
            data_sub_list.append(datetime.strptime('1000-01-01',"%Y-%m-%d"))
### Take latest file            
    m = max(data_sub_list)
    index_out = [i for i, j in enumerate(data_sub_list) if j == m]

    print("The index of the latest data is: " + str(index_out))

    data_load_string = data_list[0]
    data_in = pd.read_csv(filepath_or_buffer = r'C:\Users\hugo.lu\repos\repos_backup\ib_insync\Raw data\\' + p_contract + "\\" + data_load_string

                         ,index_col=0)
    ### Convert date column to datetime64
    data_in['date'] = data_in['date'].astype(str) 
    data_in['date']=data_in['date'].apply(convert_string_to_datetime)



string_list = [(p_file_path+today),p_contract,p_whatToShow,p_durationStr,(p_barSizeSetting+".csv")]
end_file_path = '_'.join(string_list)
end_file_path

Nothing in directory


'C:\\Users\\hugo.lu\\repos\\repos_backup\\ib_insync\\Raw data\\\\EURUSD\\2020-05-05_EURUSD_MIDPOINT_10 Y_1 day.csv'

### Connect to IBKR

In [6]:


util.startLoop()
ib = IB()
# add this line of code to debug 
#util.logToConsole(logging.DEBUG)
ib.connect('127.0.0.1', 7497, clientId=999)

<IB connected to 127.0.0.1:7497 clientId=999>

### Fetch data

In [7]:
contract = Forex(str(p_contract))

bars = ib.reqHistoricalData(
        contract,
        endDateTime= p_endDateTime ,
        durationStr=p_durationStr,
        barSizeSetting= p_barSizeSetting,
        whatToShow=p_whatToShow,
        useRTH=p_useRTH,
        formatDate=p_formatDate,
        keepUpToDate=p_keepUpToDate)
# convert to pandas dataframe (one off conversion):
#df = util.df(bars)
#print(df)

In [8]:
bars.updateEvent += onBarUpdate
ib.sleep(0.1)

True

In [9]:

ib.cancelHistoricalData(bars)

Error 162, reqId 3: Historical Market Data Service error message:API historical data query cancelled: 3


In [10]:
## Specify data out
try:
    data_out = df.copy(deep  = True)
except:
    df = util.df(bars)
    data_out = df.copy(deep = True)

data_out['date'] = data_out['date'].astype(str)
data_out['date']=data_out['date'].apply(convert_string_to_datetime)

### Save data

In [12]:

if len(data_list) == 0:
    data_out.to_csv(path_or_buf = end_file_path)
else:
    data_in_copy = data_in.copy(deep = True)
    data_in_max_date = data_in['date'].max()
    data_out_min_date = data_out['date'].min()
    data_out_max_date = data_out['date'].max()
    append_data = data_out[data_out['date'] > data_in_max_date]
    if data_out_min_date > data_in_max_date:
        print("We are missing weeks")
    elif len(append_data) == 0:
        print("Data is up to date")
        
    else:
        print("Appending data from " + str(data_in_max_date) + " to " + str(data_out_max_date))
        data_in_copy = data_in_copy.append(append_data)
        data_in_copy.reset_index(drop = True, inplace= True)
        print("Saving data")
        data_in_copy.to_csv(path_or_buf = end_file_path)

## Disconnect

In [13]:
ib.disconnect()